In [1]:
#%matplotlib inline

import xarray as xr
import numpy as np
import pandas as pd

import os.path
from datetime import datetime, timedelta, date
import time

import matplotlib.pyplot as plt
#import proplot as pplt

import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
import cartopy.feature as cfeature

from subx_utils import *

In [2]:
### TESTING ENVIRONMENT -- remove
import sys
print(sys.executable)
print (sys.version)
!conda list xarray
!conda list netcdf4

/home/mlavoie/miniconda3/envs/SubX/bin/python
3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]
# packages in environment at /home/mlavoie/miniconda3:
#
# Name                    Version                   Build  Channel
# packages in environment at /home/mlavoie/miniconda3:
#
# Name                    Version                   Build  Channel


In [3]:
# Get Start Time
start_time = time.time()

In [4]:
# Eliminate Warnings
import warnings
warnings.filterwarnings("ignore")

In [5]:
# Set xarray to keep attributes
xr.set_options(keep_attrs=True)  

### Models and Forecast Settings

In [8]:
subxmodels_list,_,_,_,_=initSubxModels()
model_labels=[item['group']+'-'+item['model'] for item in subxmodels_list]
nweeks=4

### File paths

In [9]:
url='http://iridl.ldeo.columbia.edu/SOURCES/.Models/.SubX/'
datatype='forecast'
hcstPath='/share/scratch/kpegion/subx/hindcast/'

### Fcst Date Handling

In [10]:
# Fcst Date Handling
fcstdate,fcst_week_dates=getFcstDates()
fcstdate,fcst_week_dates=getFcstDates(date='20230615')

Using Current Date:  2023-06-21 00:00:00
USING FCSTS ICS FROM:  20230609  to  20230615
Output files and figures will be labeled as:  20230615
Using Input Date:  20230615
Using Input Date:  2023-06-15 00:00:00
USING FCSTS ICS FROM:  20230609  to  20230615
Output files and figures will be labeled as:  20230615


### Make a subx_fcst `xarray.Dataset` containing all models + MME for weeks 1-4

In [11]:
print('PROCESSING FCSTS FOR: ')

# Loop over all the SubX Models
ds_models_list=[]
for imodel,subx_model in enumerate(subxmodels_list):
    
    # Get the model, group, variables, and levels from the dictionary 
    varnames=subx_model['varnames']
    plevstrs=subx_model['plevstrs']
    model=subx_model['model']
    group=subx_model['group']
    
    print('===> '+group+'-'+model)
    
    # Loop over variables for this model
    ds_anoms_list=[]
    for varname,plevstr in zip(varnames,plevstrs):
        
        # Read Data
        baseURL=url+'.'+group+'/.'+model+'/.'+datatype+'/.'+varname
        inFname=baseURL+'/'+str(date.toordinal(fcstdate))+'/pop/dods'  
        print(inFname)
        ds=xr.open_dataset(inFname)

        # Check if P is a coordinate and if so, select the Pressure Level
        if ('P' in list(ds[varname].dims)):
            ds=ds.sel(P=int(plevstr))
        
        # Remove hours,minutes,seconds from Initial Condition date
        ds['S']=ds['S'].dt.floor('d')
        
        # Identify dates in this forecast week
        dates_mask=np.isin(np.array(fcst_week_dates).astype('datetime64[ns]'),ds['S'])
        startdates=fcst_week_dates[dates_mask]
        
        # Make string versions of the dates in this forecast week
        sdates=startdates.strftime(('%Y%m%d'))
        mmdd=startdates.strftime(('%m%d'))
        
        # Select only the startdates in this fcst week
        ds=ds.sel(S=ds['S'].isin(np.array(startdates.values).astype('datetime64[ns]'))) 
        
        # Make sure there is data for this forecast week and this model, if not, skip
        if (ds['S'].size>0):
            
            # Drop any startdates with all missing
            ds=ds.dropna('S',how='all')
            
            # Make sure there is data for this forecast week and this model after dropping missing
            if (ds['S'].size>0):
                
                # Select the most recent available start date in this forecast week
                if group=='NRL': 
                    ds=preprocNRL(ds,startdates)
                elif group=='NCEP':
                    ds=preprocCFSv2(ds)
                else:
                    # Select the most recent non-missing start date in fcst week
                    ds=ds.sel(S=ds['S'][-1])
                    
                    # Get Subsetted Data Using Ingrid
                    ds=getDataViaIngrid(ds,baseURL)
                    
                    # Save the start date information to ic_dates
                    ds=ds.assign_coords({'ic_dates':pd.to_datetime(ds['S'].values).strftime('%Y%m%d')})

                # Create model as coordinates to keep track of which models are being used
                ds=ds.assign_coords({'model':group+'-'+model})    
                        
                # L got treated as a timeDelta on read in; convert to datetime64 based on start date
                ds['L']=ds['S'].values+ds['L'].dt.floor('d')
                
                # Rename coordinates & add attributes
                ds_out=ds.rename({'X':'lon','Y':'lat','L':'lead'})  
                ds_out['lon'].attrs['units']='degrees_east'
                ds_out['lat'].attrs['units']='degrees_north'
            
                # Get Climo and reset and re-name coordinates as needed to subtract from full field
                climo_fname=hcstPath+varname+plevstr+'/daily/climo/'+group+'-'+model+'/'+varname+'_'+group+'-'+model+'_'+mmdd[-1]+'.climo.p.nc'
                ds_clim=xr.open_dataset(climo_fname)
                ds_clim=ds_clim.rename({'time':'lead'})
                ds_clim['lead']=ds['L']
            
                # Special handling for GEFSv12 incorrect units for precip
                if (model=='GEFSv12' and varname=='pr'):
                    ds_clim=ds_clim/86400.0
                    
                # Special handling for GEFSv12_CPC  incorrect units for precip
                if (model=='GEFSv12_CPC' and varname=='pr'):
                    ds_clim=ds_clim/86400.0
            
                # Calculate Ensemble Mean and Ensemble Mean Anomalies       
                ds_emean=ds_out.mean(dim='M').squeeze() 
                ds_anoms=ds_emean[varname]-ds_clim[varname]
            
                # Assign the number of ensemble members
                ds_anoms['nens']=len(ds['M'])
                
                # Append anoms to list
                ds_anoms_list.append(ds_anoms)
                
            else:
                # Report that this model is missing
                print(model+' '+varname+' is missing for forecast date: ',fcstdate) 
        else:
            # Report that this model is missing
            print(model+' '+varname+' is missing for forecast date: ',fcstdate) 
    
    # Make list with all variables for this model and append to models list 
    if (ds_anoms_list):
        ds_models_list.append(xr.merge(ds_anoms_list))
    

# Combine into dataset with all variables and all models
ds_models=xr.combine_nested(ds_models_list,concat_dim='model').persist()

# Make Weekly Averages Consistent with CPC Forecast Week of Sat-Fri
ds_week=makeFCSTWeeks(ds_models,fcstdate,nweeks)

# Make SubX-MME
ds_mme=ds_week.mean(dim='model')
ds_mme=ds_mme.assign_coords({'S':fcstdate,'model':'SUBX-MME',
                             'nens':ds_week['nens'].sum(),
                             'ic_dates':pd.to_datetime(fcstdate).strftime('%Y%m%d')})

# Combine dataset for individual models and MME into a single xarray.Dataset 
ds_subx_fcst=xr.concat([ds_week,ds_mme],dim='model').compute()

# Set global attributes
ds_subx_fcst=setattrs(ds_subx_fcst,fcstdate)
print()
print("SUBX FCST DATASET: ")
print(ds_subx_fcst)

# Write Files
print()
print("WRITING DATA")
#subxWrite(ds_subx_fcst,fcstdate)

# Make Figures
#figpath='/shared/subx/forecast/weekly/'+fcstdate.strftime('%Y%m%d')+'/images/'
figpath='/share/scratch/mlavoie/subx/figs_test/'
print()
print("MAKING FIGURES")
subxPlot(ds_subx_fcst,figpath)

PROCESSING FCSTS FOR: 
===> ESRL-FIMr1p1
http://iridl.ldeo.columbia.edu/SOURCES/.Models/.SubX/.ESRL/.FIMr1p1/.forecast/.ua/738686/pop/dods
850


FileNotFoundError: [Errno 2] No such file or directory: b'/share/scratch/kpegion/subx/hindcast/ua850/daily/climo/ESRL-FIMr1p1/ua_ESRL-FIMr1p1_0615.climo.p.nc'

In [ ]:
# Print timing information
print()
print("--- %s seconds ---" % (time.time() - start_time))